<a href="https://colab.research.google.com/github/Simon-Pu/Temp/blob/master/EfficientNet_PyTorch_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!nvidia-smi

In [0]:
import pynvml

pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
device_name = pynvml.nvmlDeviceGetName(handle)

if device_name != b'Tesla T4' and device_name != b'Tesla P100-PCIE-16GB':
  raise Exception("""
    Unfortunately this instance does not have a T4 GPU.   
    Please make sure you've configured Colab to request a GPU instance type.
    Sometimes Colab allocates a Tesla K80 instead of a T4. Resetting the instance.
    If you get a K80 GPU, try Runtime -> Reset all runtimes...
  """)
else:
  print('Woo! You got the right kind of GPU!:::', device_name)

In [0]:
# Download image and class labels
!wget https://raw.githubusercontent.com/lukemelas/EfficientNet-PyTorch/master/examples/simple/img.jpg
!wget https://raw.githubusercontent.com/lukemelas/EfficientNet-PyTorch/master/examples/simple/labels_map.txt

In [0]:
# Get EfficientNet PyTorch
#!pip install efficientnet_pytorch
# Get PyTorch Image Models, etc
# https://github.com/rwightman/pytorch-image-models
!pip install timm

In [0]:
import json
from PIL import Image
import numpy as np

import torch
import torchvision
import torchvision
import torchvision.transforms as transforms
import torch.backends.cudnn as cudnn
from torch.autograd import Variable
#from torchvision import transforms

#from efficientnet_pytorch import EfficientNet
import timm

In [0]:
seed = 0
gpu = 0

np.random.seed(seed)
torch.cuda.set_device(gpu)
cudnn.benchmark = True
torch.manual_seed(seed)
cudnn.enabled = True
torch.cuda.manual_seed(seed)

model_name = 'efficientnet-b0'
#image_size = EfficientNet.get_image_size(model_name) # 224
image_size = 224

In [0]:
img = Image.open('img.jpg')
#img

In [0]:
# Preprocess image
tfms = transforms.Compose([transforms.Resize(image_size), transforms.CenterCrop(image_size), 
                           transforms.ToTensor(),
                           transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),])
img = tfms(img).unsqueeze(0)
img = Variable(img)
img = img.cuda()

In [0]:
# Load class names
labels_map = json.load(open('labels_map.txt'))
labels_map = [labels_map[str(i)] for i in range(1000)]

In [0]:
# Classify with EfficientNet
#model = EfficientNet.from_pretrained(model_name)
#model = model.cuda()
#model.cuda()
#model.eval()
#with torch.no_grad():
#    logits = model(img)
#preds = torch.topk(logits, k=5).indices.squeeze(0).tolist()

#print('-----')
#for idx in preds:
#    label = labels_map[idx]
#    prob = torch.softmax(logits, dim=1)[0, idx].item()
#    print('{:<75} ({:.2f}%)'.format(label, prob*100))

# How to profiling layer-by-layer in Pytroch?
#with torch.autograd.profiler.profile(use_cuda=True) as prof:
#    model(img)
#print(prof) 

In [0]:
# Classify with EfficientNet
# https://github.com/rwightman/pytorch-image-models
model = timm.create_model('efficientnet_b0', pretrained=True)
#model = EfficientNet.from_pretrained(model_name)
model = model.cuda()
#model.cuda()
model.eval()
with torch.no_grad():
    logits = model(img)
preds = torch.topk(logits, k=5).indices.squeeze(0).tolist()

print('-----')
for idx in preds:
    label = labels_map[idx]
    prob = torch.softmax(logits, dim=1)[0, idx].item()
    print('{:<75} ({:.2f}%)'.format(label, prob*100))

# How to profiling layer-by-layer in Pytroch?
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    model(img)
print(prof) 

In [0]:
!wget https://s3.amazonaws.com/fast-ai-imageclas/imagewoof.tgz
!tar zxvf imagewoof.tgz

In [0]:
#!ls ./imagewoof/val/

In [0]:
import argparse
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.multiprocessing as mp
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.nn import Linear

# define loss function (criterion) and optimizer
lr = 0.1
momentum = 0.9
weight_decay = 1e-4
print_freq = 50
start_epoch = 0
epochs = 10
gpu = 0

model = timm.create_model('efficientnet_b0', pretrained=True)
#print(model)
#classifier): Linear(in_features=1280, out_features=1000, bias=True)
model.classifier = Linear(in_features=1280, out_features=10, bias=True)
#model = EfficientNet.from_pretrained(model_name)
model = model.cuda()

criterion = nn.CrossEntropyLoss().cuda(gpu)
optimizer = torch.optim.SGD(model.parameters(), lr,
                            momentum=momentum,
                            weight_decay=weight_decay)

In [0]:
# Data loading code
traindir = os.path.join('./imagewoof/train')
valdir = os.path.join('./imagewoof/val/')
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
     traindir,
     transforms.Compose([
         transforms.RandomResizedCrop(224),
         transforms.RandomHorizontalFlip(),
         transforms.ToTensor(),
         normalize,
     ]))

train_sampler = None
batch_size = 32

train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=(train_sampler is None),
    pin_memory=True, sampler=train_sampler)

val_loader = torch.utils.data.DataLoader(
    datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ])),
    batch_size=batch_size, shuffle=False,
    pin_memory=True)

In [0]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)


class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'


def adjust_learning_rate(optimizer, epoch, lr):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = lr * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr


def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [0]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    # switch to train mode
    model.train()

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        #if args.gpu is not None:
        images = images.cuda(0, non_blocking=True)
        target = target.cuda(0, non_blocking=True)

        # compute output
        output = model(images)
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            progress.display(i)

In [0]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            #if args.gpu is not None:
            images = images.cuda(0, non_blocking=True)
            target = target.cuda(0, non_blocking=True)

            # compute output
            output = model(images)
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % print_freq == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg


In [0]:
best_acc1 = 0.0
for epoch in range(start_epoch, epochs):

   adjust_learning_rate(optimizer, epoch, lr)

   # train for one epoch
   train(train_loader, model, criterion, optimizer, epoch)

   # evaluate on validation set
   acc1 = validate(val_loader, model, criterion)

   # remember best acc@1 and save checkpoint
   is_best = acc1 > best_acc1
   best_acc1 = max(acc1, best_acc1)
   #print (best_acc1)

print ('Finish')

In [0]:
data_sub = './imagewoof/val/'
batch_size = 1
test_transform = transforms.Compose([
            transforms.Resize(224, interpolation=Image.BICUBIC),
            transforms.ToTensor(),
        ])        
test_data = torchvision.datasets.ImageFolder(root=data_sub, transform=test_transform)

num_test = len(test_data)
print (num_test)
test_queue = torch.utils.data.DataLoader(
            test_data,
            batch_size=batch_size,
            shuffle=False,
            pin_memory=1)

In [0]:
confusion_matrix = torch.zeros(10, 10)
with torch.no_grad():

   for step, (input, target) in enumerate(test_queue):
       input = Variable(input).cuda()
       target = Variable(target).cuda(async=True)
       #input = Variable(input)
       #target = Variable(target)

       outputs = model(input)
       _, preds = torch.max(outputs, 1)
       #print(preds)
       for t, p in zip(target.view(-1), preds.view(-1)):
           #print(t, p)
           confusion_matrix[t.long(), p.long()] += 1
#print(confusion_matrix)
#print(confusion_matrix.diag()/confusion_matrix.sum(1))

In [0]:
#print(target.view(-1))
#print(preds.view(-1))
#print(confusion_matrix[0,155]>=1)
#for t in range (10):
# for p in range (10): 
#  if confusion_matrix[t,p]>=1:
#   print(t, p, confusion_matrix[t,p])

import pandas as  pd
import seaborn as sn
import matplotlib.pyplot as plt

px = confusion_matrix.numpy()
px = pd.DataFrame(px)
sn.set(font_scale=1.4) # for label size
sn.heatmap(px, annot=True, annot_kws={"size": 16}) # font size

plt.show()

In [0]:
import pandas as  pd
import seaborn as sn
import matplotlib.pyplot as plt
import torch
confusion_matrix = torch.zeros(10, 10)
# 36, 8
confusion_matrix = torch.tensor([[48.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.],
        [ 0., 49.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  0., 48.,  1.,  0.,  1.,  0.,  0.,  0.,  0.],
        [ 0.,  2., 12., 33.,  0.,  0.,  1.,  0.,  0.,  2.],
        [ 0.,  2.,  0.,  0., 48.,  0.,  0.,  0.,  0.,  0.],
        [ 3.,  0.,  0.,  0.,  1., 44.,  1.,  1.,  0.,  0.],
        [ 0.,  0.,  1.,  0.,  0.,  0., 47.,  0.,  0.,  2.],
        [ 1.,  0.,  0.,  0.,  0.,  0.,  0., 49.,  0.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 50.,  0.],
        [ 1.,  1.,  2.,  0.,  1.,  0.,  0.,  0.,  0., 45.]])
# 24, 8
confusion_matrix = torch.tensor([[47.,  0.,  0.,  0.,  2.,  1.,  0.,  0.,  0.,  0.],
        [ 0., 49.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
        [ 0.,  1., 45.,  4.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1., 13., 33.,  0.,  0.,  1.,  0.,  0.,  1.],
        [ 0.,  1.,  1.,  0., 48.,  0.,  0.,  0.,  0.,  0.],
        [ 2.,  0.,  0.,  0.,  3., 42.,  0.,  2.,  0.,  1.],
        [ 0.,  0.,  1.,  0.,  1.,  0., 47.,  0.,  0.,  1.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0., 49.,  1.,  0.],
        [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 50.,  0.],
        [ 0.,  2.,  1.,  2.,  1.,  0.,  0.,  1.,  0., 43.]])
px = confusion_matrix.numpy()
px = pd.DataFrame(px)
sn.set(font_scale=1.4) # for label size
sn.heatmap(px, annot=True, annot_kws={"size": 16}) # font size

plt.show()
print(confusion_matrix.diag()/confusion_matrix.sum(1))
TP = confusion_matrix.diag() 
print('TP :', confusion_matrix.diag())
Total= confusion_matrix.sum(1) 
print(confusion_matrix.sum(1))
print('Total :', Total.sum(0))
FP = confusion_matrix.sum(0)-confusion_matrix.diag()
print('FP :', confusion_matrix.sum(0)-confusion_matrix.diag())
FN = confusion_matrix.sum(1)-confusion_matrix.diag()
print('FN :', confusion_matrix.sum(1)-confusion_matrix.diag())
TN = Total.sum(0)- TP - FP - FN
print('TN :', Total.sum(0)- TP - FP - FN)
Precision = TP/(TP+FP)
print('Precision(TP/(TP+FP)):', confusion_matrix.diag()/(confusion_matrix.diag()+( confusion_matrix.sum(0) - confusion_matrix.diag() )))
Recall = TP/(TP+FN)
print('Recall(TP/(TP+FN)) :', confusion_matrix.diag()/(confusion_matrix.diag()+( confusion_matrix.sum(1) - confusion_matrix.diag() )))
Accuracy = (TP+TN)/(TP+FN+FP+TN)
print('Accuracy(TP+TN/(TP+FN+FP+TN)) :',(TP+TN)/(TP+FN+FP+TN))
F1 = (2/((1/Precision)+(1/Recall)))
print('F1 Score(2/(1/Precision)+(1/Recall)) :', (2/((1/Precision)+(1/Recall))))
print('-----------------------------------------------------------------------')
print('Finial Precision:', Precision.sum(0)/10)
print('Finial Recall   :', Recall.sum(0)/10)
print('Finial Accuracy :', Accuracy.sum(0)/10)
print('Finial F1 Score :', F1.sum(0)/10)